Run Iterations all at Once

In [1]:
import importlib

import metaspread
import metaspread.simrunner as simrunner
import metaspread.configs

importlib.reload(metaspread)

import pandas as pd
import random
import time
import os
import ast
import json
import uuid

def run_n_times(number_of_simulations, maximum_steps):
    # Set random seed for reproducibility
    random.seed(int(time.time()))

    # Get the directory where metaspread is installed
    package_dir = os.path.dirname(metaspread.__file__)

   # Define paths for the default and simulation configuration files
    default_config_path = os.path.join(package_dir, "default_configs.csv")
    simulation_config_path = os.path.join(package_dir, "simulations_configs.csv")
    # print(simulation_config_path)

    # Load default parameters from default_configs.csv (This file remains unchanged)
    df_default_configs = pd.read_csv(default_config_path, header=0, converters={"Values": ast.literal_eval})
    # print("default configs")
    # display(df_default_configs.head())
    default_params = dict(zip(df_default_configs["Names"], df_default_configs["Values"]))

    # Load simulation configuration file
    df_simulation_configs = df_default_configs.copy()  # Start with defaults
    
    # List of parameters to vary
    user_selected_params = ["dM", "dE", "phiM", "phiE", "mesenchymal_proportion", "epithelial_proportion"]
    valid_params = [param for param in user_selected_params if param in default_params]

    # Number of simulations
    num_simulations = number_of_simulations

    # Store all parameter variations
    simulation_logs = []

    # Loop over multiple iterations
    for i in range(num_simulations):
        print(f"Running simulation {i+1}/{num_simulations}...")

        simulation_id = str(uuid.uuid4())[:8]

        # Modify parameters randomly
        varied_params = default_params.copy()
        for param in valid_params:
            if param != "mesenchymal_proportion" and param != "epithelial_proportion":
                default_value = float(default_params[param])  # Ensure it's numeric
                new_value = default_value * random.uniform(1/3, 3)
                # print(f"param: {param} new value: {new_value}")
                varied_params[param] = new_value

        # Ensure grid size is always set to 50
        varied_params["gridsize"] = 100

        # set mesenchymal_porpotion and ensure the proportions add up to 1
        mes_proportion = random.uniform(0,1)
        varied_params["mesenchymal_proportion"] = mes_proportion
        varied_params["epithelial_proportion"] = 1 - mes_proportion


        # print("varied_params")
        # print(varied_params)

        # Update simulations_configs.csv but NOT default_configs.csv
        for param in valid_params + ["gridsize"]:
            if param in df_simulation_configs["Names"].str.strip().values:  # Check if param exists
                df_simulation_configs.loc[df_simulation_configs["Names"].str.strip() == param, "Values"] = varied_params[param]
                print(f"Updated {param} to {varied_params[param]}")
            else:
                print(f"⚠️ Warning: {param} not found in df_simulation_configs[\"Names\"]. Skipping update.")

        df_simulation_configs.to_csv(simulation_config_path, index=False, mode='w')
        # print("df_simuation_configs")
        # display(df_simulation_configs.head(10))

        # df_simulation_configs = pd.read_csv(os.path.join(package_dir, "simulations_configs.csv"))
        # print("csv file")
        # display(df_simulation_configs.head(10))

        # Run simulation
        simulation = simrunner.run_simulation(simulation_id, max_steps=maximum_steps, data_collection_period=100)

        # Store parameter variations
        simulation_logs.append({
            "simulation_id": simulation_id,
            "varied_parameters": {param: varied_params[param] for param in valid_params},
            "gridsize": 100
        })

        print(f"Completed simulation {i+1}/{num_simulations}.")

    # Save all parameters used in simulations
    with open("metaspread_simulation_logs.json", "w") as f:
        json.dump(simulation_logs, f, indent=4)

    print(f"All {num_simulations} simulations completed successfully.")


## Run
run_n_times({number of simulations}, {maximum steps})

In [2]:
run_n_times(5,2160)

Running simulation 1/5...
Updated dM to 0.000280790546133908
Updated dE to 8.946223002616861e-05
Updated phiM to 0.0010339462076662076
Updated phiE to 0.0004216287343515398
Updated mesenchymal_proportion to 0.43035113531934743
Updated epithelial_proportion to 0.5696488646806526
Updated gridsize to 100
	 Folder for this simulation: Simulations/Sim-42eab642-Date-2025-03-02_01-23-51
	 Saving agents data at: Simulations/Sim-42eab642-Date-2025-03-02_01-23-51
	 Saving Mmp2 data at: Simulations/Sim-42eab642-Date-2025-03-02_01-23-51/Mmp2
	 Saving Ecm data at: Simulations/Sim-42eab642-Date-2025-03-02_01-23-51/Ecm
	 Saving Vasculature data at: Simulations/Sim-42eab642-Date-2025-03-02_01-23-51/Vasculature
	 Saving all the simulations parameters at: Simulations/Sim-42eab642-Date-2025-03-02_01-23-51/configs.csv
Starting simulation from zero!
Finished the simulation at time step 10!
Completed simulation 1/5.
Running simulation 2/5...
Updated dM to 0.00023939812263687658
Updated dE to 0.0001327363966

## For the data and graphics generation
⚠️ Still working on this part

In [ ]:
import metaspread.datagenerator as datagenerator
import metaspread.graphgenerator as graphgenerator

In [26]:
simulation_folder = "Sim-6120b2be-Date-2025-03-01_16-54-53"
amount_of_pictures= 50

datagenerator.generate_data(simulation_folder)

	Analyzing data in the folder Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53

Using cells data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/CellsData.csv
Using Ecm data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Ecm
Using Mmp2 data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Mmp2
Using vasculature data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Vasculature
Max step reached is 500 while 500 was expected.

Saving tumor data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Data analysis/Tumor dynamics
Saving cells numbers data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Data analysis/Cells growth
Saving vasculature data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Data analysis/Vasculature dynamics

Grid: 1
	Saving tumor data...
	Saving cells numbers graph data...

Grid: 2
	Saving tumor data...
	Saving cells numbers graph data...

Grid: 3
	Saving tumor data...
	Saving

In [27]:
graphgenerator.generate_graphs(simulation_folder, amount_of_pictures)

Analyzing data in the folder Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53

Using cells data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/CellsData.csv
Using Ecm data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Ecm
Using Mmp2 data in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Mmp2
Using vasculature data at: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Vasculature
Max step reached is 500 while 500 was expected.

Saving tumor images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Tumor dynamics
Saving Mmp2 images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Mmp2 dynamics
Saving Ecm images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Ecm dynamics
Saving cells numbers images in the folder: Simulations/Sim-6120b2be-Date-2025-03-01_16-54-53/Graphical analysis/Cells growth
Saving vasculature images in the folder: Simulation